# build a package installed container as it used base image

In [6]:
%%writefile Docker/newimage/Dockerfile
FROM gcr.io/deeplearning-platform-release/tf-cpu.1-15
RUN git clone https://github.com/mingxuts/multi-center-fed-learning
WORKDIR multi-center-fed-learning
RUN pip install --no-cache-dir pot
RUN pip install cmake
RUN pip install --no-cache-dir -r requirements.txt

Writing Docker/newimage/Dockerfile


In [ ]:
#!git clone https://github.com/mingxuts/multi-center-fed-learning

In [3]:
!cp ../multi-center-fed-learning/requirements.txt Docker/newimage/

In [5]:
%%writefile Docker/newimage/.dockerignore
Dockerfile
README.md
*.pyc
*.pyo
*.pyd
__pycache__
.pytest_cache

Writing Docker/newimage/.dockerignore


In [ ]:
!gcloud container images list \
  --repository="gcr.io/deeplearning-platform-release"

In [ ]:
!gcloud builds submit --tag gcr.io/robust-multicenter-fl/base newimage

# Build docker for dataset femnist

In [51]:
%%writefile Docker/data_fem/data_fem.bash
#!/usr/bin/env bash

echo "Transfering data to local..."
mkdir -p ~/leaf/data/femnist/data/test
mkdir -p ~/leaf/data/femnist/data/train
gsutil -m rsync -r gs://gs_bak/leaf/data/femnist/data/test ~/leaf/data/femnist/data/test
gsutil -m rsync -r gs://gs_bak/leaf/data/femnist/data/train ~/leaf/data/femnist/data/train

echo "Finished."

Overwriting Docker/data_fem/data_fem.bash


In [52]:
%%writefile Docker/data_fem/Dockerfile
FROM gcr.io/robust-multicenter-fl/base

COPY ./data_fem.bash /root
RUN ["chmod", "+x", "/root/data_fem.bash"]
RUN ["/root/data_fem.bash"]

Overwriting Docker/data_fem/Dockerfile


In [1]:
%%writefile Docker/train_fem/training.bash
#!/usr/bin/env bash
function sync_history() {
    pushd ./metrics
    last_f=`ls -lt | awk '/history/ {print $NF}'`
    echo "$1${last_f}"
    gsutil -m cp "./${last_f}" "gs://gs_bak/mcflhistory/$1${last_f}"
    rm "./${last_f}"
    popd
}


echo "****** Experiment to run on FEMNIST ******"
cd /multi-center-fed-learning
git pull origin rbmcfl
cd /multi-center-fed-learning/models
DATASET="FEMNIST"

python experiment.py -dataset femnist -experiment fedavg
FILE="$DATASET-fedavg-"
sync_history FILE

python experiment.py -dataset femnist -experiment fedrobust
FILE="$DATASET-fedrobust-"
sync_history FILE

python experiment.py -dataset femnist -experiment fedsem
FILE="$DATASET-fedsem-"
sync_history FILE

Overwriting Docker/train_fem/training.bash


In [2]:
%%writefile Docker/train_fem/training_poison.bash
#!/usr/bin/env bash
function sync_history() {
    pushd ./metrics
    last_f=`ls -lt | awk '/history/ {print $NF}'`
    echo "$1${last_f}"
    gsutil -m cp "./${last_f}" "gs://gs_bak/mcflhistory/$1${last_f}"
    rm "./${last_f}"
    popd
}

cd /multi-center-fed-learning
git pull origin rbmcfl
cd /multi-center-fed-learning/models
DATASET="FEMNIST"

echo "****** Experiment to run on Femnist ******"
python experiment.py -dataset femnist -experiment fedavg -configuration job-poison.yaml
FILE="$DATASET-fedavg-poison-"
sync_history FILE

python experiment.py -dataset femnist -experiment fedrobust -configuration job-poison.yaml
FILE="$DATASET-fedsem-poison-"
sync_history FILE

python experiment.py -dataset femnist -experiment fedsem -configuration job-poison.yaml
FILE="$DATASET-fedrobust-poison-"
sync_history FILE

Overwriting Docker/train_fem/training_poison.bash


In [3]:
%%writefile Docker/train_fem/Dockerfile
FROM gcr.io/robust-multicenter-fl/data_fem

WORKDIR /root
COPY ./training.bash /root
COPY ./training_poison.bash /root
RUN chmod +x ./training.bash
RUN chmod +x ./training_poison.bash

Overwriting Docker/train_fem/Dockerfile


In [ ]:
#!gcloud builds submit --tag gcr.io/robust-multicenter-fl/train_fem_op Docker/train_fem

# Build docker for celeba dataset

In [56]:
%%writefile Docker/data_cel/data_cel.bash
#!/usr/bin/env bash

echo "Transfering data to local..."
mkdir -p ~/leaf/data/celeba/data/raw
mkdir -p ~/leaf/data/celeba/data/test
mkdir -p ~/leaf/data/celeba/data/train
gsutil -m rsync -r gs://gs_bak/leaf/data/celeba/data/raw ~/leaf/data/celeba/data/raw
gsutil -m rsync -r gs://gs_bak/leaf/data/celeba/data/test ~/leaf/data/celeba/data/test
gsutil -m rsync -r gs://gs_bak/leaf/data/celeba/data/train ~/leaf/data/celeba/data/train

echo "Transfer done"

Writing Docker/data_cel/data_cel.bash


In [57]:
%%writefile Docker/data_cel/Dockerfile
FROM gcr.io/robust-multicenter-fl/base

WORKDIR /root
COPY ./data_cel.bash /root
RUN chmod +x ./data_cel.bash
RUN ./data_cel.bash

Writing Docker/data_cel/Dockerfile


In [58]:
%%writefile Docker/train_cel/training.bash
#!/usr/bin/env bash
function sync_history() {
    pushd ./metrics
    last_f=`ls -lt | awk '/history/ {print $NF}'`
    echo "$1${last_f}"
    gsutil -m cp "./${last_f}" "gs://gs_bak/mcflhistory/$1${last_f}"
    rm "./${last_f}"
    popd
}

echo "****** Experiment to run on Celeba ******"

cd /multi-center-fed-learning
git pull origin rbmcfl
cd /multi-center-fed-learning/models
DATASET="CELEBA"
python experiment.py -dataset celeba -experiment fedavg
FILE="$DATASET-fedavg-"
sync_history FILE

python experiment.py -dataset celeba -experiment fedsem
FILE="$DATASET-fedsem-"
sync_history FILE

python experiment.py -dataset celeba -experiment fedrobust
FILE="$DATASET-fedrobust-"
sync_history FILE

echo "Training Data copied to bucket, job successfully completed. "

Writing Docker/train_cel/training.bash


In [59]:
%%writefile Docker/train_cel/training_poison.bash
#!/usr/bin/env bash
function sync_history() {
    pushd ./metrics
    last_f=`ls -lt | awk '/history/ {print $NF}'`
    echo "$1${last_f}"
    gsutil -m cp "./${last_f}" "gs://gs_bak/mcflhistory/$1${last_f}"
    rm "./${last_f}"
    popd
}

cd /multi-center-fed-learning
git pull origin rbmcfl
cd /multi-center-fed-learning/models
DATASET="CELEBA"
echo "****** Experiment to run on Celeba ******"
python experiment.py -dataset celeba -experiment fedavg -configuration job-poison.yaml
FILE="$DATASET-fedavg-poison-"
sync_history FILE

python experiment.py -dataset celeba -experiment fedsem -configuration job-poison.yaml
FILE="$DATASET-fedsem-poison-"
sync_history FILE

python experiment.py -dataset celeba -experiment fedrobust -configuration job-poison.yaml
FILE="$DATASET-fedrobust-poison-"
sync_history FILE
echo "Training result copied to bucket, job successfully completed. "

Writing Docker/train_cel/training_poison.bash


In [60]:
%%writefile Docker/train_cel/Dockerfile
FROM gcr.io/robust-multicenter-fl/data_op

WORKDIR /root
COPY ./training.bash /root
COPY ./training_poison.bash /root
RUN chmod +x ./training.bash
RUN chmod +x ./training_poison.bash

Writing Docker/train_cel/Dockerfile


# Build docker file for Synthetic dataset

In [61]:
%%writefile Docker/train_syn/preprocess.bash
#!/usr/bin/env bash

cd /root/leaf/data/synthetic
python main.py -num-tasks 1000 -num-classes 5 -num-dim 60
./preprocess.sh -s niid --sf 1.0 -k 5 -t sample --tf 0.6

Writing Docker/train_syn/preprocess.bash


In [62]:
%%writefile Docker/train_syn/training.bash
#!/usr/bin/env bash
function sync_history() {
    pushd ./metrics
    last_f=`ls -lt | awk '/history/ {print $NF}'`
    echo "$1${last_f}"
    gsutil -m cp "./${last_f}" "gs://gs_bak/mcflhistory/$1${last_f}"
    rm "./${last_f}"
    popd
}

echo "****** Experiment to run on Celeba ******"

cd /multi-center-fed-learning
git pull origin rbmcfl
cd /multi-center-fed-learning/models
DATASET="SYNTHETIC"
python experiment.py -dataset synthetic -experiment fedavg
FILE="$DATASET-fedavg-"
sync_history FILE

python experiment.py -dataset synthetic -experiment fedsem
FILE="$DATASET-fedsem-"
sync_history FILE

python experiment.py -dataset synthetic -experiment fedrobust
FILE="$DATASET-fedrobust-"
sync_history FILE
echo "Training result copied to bucket, job successfully completed. "

Writing Docker/train_syn/training.bash


In [63]:
%%writefile Docker/train_syn/Dockerfile
FROM gcr.io/robust-multicenter-fl/base

WORKDIR /root
RUN git clone https://github.com/TalwalkarLab/leaf
COPY ./preprocess.bash ./
RUN chmod +x ./preprocess.bash
RUN ./preprocess.bash
COPY ./training.bash /root
COPY ./training_poison.bash /root
RUN chmod +x ./training.bash
RUN chmod +x ./training_poison.bash

Overwriting Docker/train_syn/Dockerfile


In [64]:
%%writefile Docker/train_syn/training_poison.bash
#!/usr/bin/env bash
function sync_history() {
    pushd ./metrics
    last_f=`ls -lt | awk '/history/ {print $NF}'`
    echo "$1${last_f}"
    gsutil -m cp "./${last_f}" "gs://gs_bak/mcflhistory/$1${last_f}"
    rm "./${last_f}"
    popd
}

cd /multi-center-fed-learning
git pull origin rbmcfl
cd /multi-center-fed-learning/models
echo "****** Experiment to run on Synthetic ******"
DATASET="SYNTHETIC"
python experiment.py -dataset synthetic -experiment fedavg -configuration job-poison.yaml
FILE="$DATASET-fedavg-poison-"
sync_history FILE

python experiment.py -dataset synthetic -experiment fedsem -configuration job-poison.yaml
FILE="$DATASET-fedsem-poison-"
sync_history FILE

python experiment.py -dataset synthetic -experiment fedrobust -configuration job-poison.yaml
FILE="$DATASET-fedrobust-poison-"
sync_history FILE
echo "Training result copied to bucket, job successfully completed. "

Writing Docker/train_syn/training_poison.bash


# Unit Test Code

In [65]:
%%writefile test1.bash
#!/usr/bin/env bash

function sync_history() {
    pushd ./metrics
    last_f=`ls -lt | awk '/history/ {print $NF}'`
    echo "$1${last_f}"
    gsutil -m cp "./${last_f}" "gs://gs_bak/mcflhistory/$1${last_f}"
    rm "./${last_f}"
    popd
}

cd /home/jupyter/multi-center-fed-learning/models
DATASET="CELEBA"
sync_history "$DATASET-fedavg-"

Overwriting test1.bash


In [66]:
#!bash test.bash

~/multi-center-fed-learning/models/metrics ~/google-mlops
writer.py
Hello, World
Hello, Worldwriter.py
~/google-mlops


In [ ]:
#!gsutil cp gs://gs_bak/mcflhistory/* ~/multi-center-fed-learning/models/metrics

In [ ]:
#!bash test1.bash